In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'

my_cmap = plt.get_cmap('gray')
#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

#Project Specific packages:
import random
import multiprocessing
from joblib import Parallel, delayed
from pywt._extensions._pywt import (DiscreteContinuousWavelet, ContinuousWavelet,
                                Wavelet, _check_dtype)
from pywt._functions import integrate_wavelet, scale2frequency
from time import time as ti
import datetime

from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
 
from sklearn.model_selection import train_test_split
  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"  #Use for GPU    
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #use for CPU

import tensorflow as tf


In [2]:

import CoreFunctions as cf


In [3]:
scales = 500
img_height , img_width = scales, 100
FrameLength = img_width
numberFrames = 600
DoSomeFiles = True
NumberOfFiles = 10
SmoothType = 1  # 0 = none, 1 = rolling average, 2 = rolling StdDev
SmoothDistance=10
TrainEpochs = 4
num_cores = multiprocessing.cpu_count() -1
defaultWavelet = 'gaus2'

#SaveModelFolder = rootfolder + 'SavedModel\\'
SaveModelFolder = cf.rootfolder + 'SavedModel/'

files = os.listdir(cf.folder)
if DoSomeFiles: files = random.sample(files,NumberOfFiles)



In [4]:

'''
def getAcceleration(FileName):
    try:
        DataSet = np.genfromtxt(open(folder+FileName,'r'), delimiter=',',skip_header=0)
        rData = getRAcceleration(DataSet[:,2:5])
        rSmoothed = Smoothing(rData)
        #return [[FileName,'x',DataSet[:,2]],[FileName,'y',DataSet[:,3]],[FileName,'z',DataSet[:,4]],[FileName,'r',rData]]
        return [FileName, 'r', rSmoothed]
    except:
        return [False,FileName,False]

'''

def makePrints(DataArray):
    
        FingerPrint = cf.getThumbprint(DataArray,defaultWavelet)
        return FingerPrint
    



In [5]:

def getResults(FPnMd):
    Ridges = cf.RidgeCount(FPnMd[2][:,500:59500])
    return [FPnMd[0],FPnMd[1],Ridges]

def CountAboveThreshold(Ridges, Threshold = 10):
    Cnum = np.count_nonzero(Ridges[2] >= Threshold)
    return [Ridges[0],Ridges[1],Cnum]


AllFingers = [] #np.asarray(dtype=object)



In [6]:
AllAccels = Parallel(n_jobs=num_cores)(delayed(cf.getAcceleration)(file) for file in files)


In [7]:
Flattened = []
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False:
        print(j,AllAccels[j][1])
    else: 
        Flattened.append(AllAccels[j][0][2])
        Flattened.append(AllAccels[j][1][2])
        Flattened.append(AllAccels[j][2][2])
print('Have Data')


Have Data


c:\Users\Dan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [8]:
Flattened = np.matrix(Flattened)

In [9]:
np.shape(Flattened)

(30, 60000)

In [10]:
Moves, files = cf.RemoveNonmovers(Flattened, files, 'X')

In [11]:
np.shape(Moves)

(5, 60000)

In [12]:
AllFingers =  Parallel(n_jobs=num_cores)(delayed(makePrints)(np.asarray(datas).flatten()) for datas in Moves)

print('Have fingerprints')


Have fingerprints


In [13]:
DataSet = Parallel(n_jobs=num_cores)(delayed(cf.ParseData)(finger) for finger in AllFingers)


In [28]:
ResultsSet = Parallel(n_jobs=num_cores)(delayed(cf.truthVector)(file) for file in files)

s = []
for r in ResultsSet:
    s.append(np.asarray(r).flatten())
ResultsSet = np.matrix(s)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(DataSet, ResultsSet, test_size=0.20, shuffle=True, random_state=0)


In [36]:
print(np.shape(X_train), np.shape(y_train))

(4, 100, 300, 250) (4, 4)


In [32]:
model = Sequential()
model.add(ConvLSTM2D(filters = 32, 
            kernel_size = (5, 5), 
            return_sequences = False, 
            data_format = "channels_last", 
            input_shape = (np.shape(X_train)[1], np.shape(X_train)[2], np.shape(X_train)[3], 1)
            )
        )
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(np.shape(y_train)[1], activation = "softmax"))
 
model.summary()
 


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_1 (ConvLSTM2D)  (None, 296, 246, 32)      105728    
                                                                 
 dropout_2 (Dropout)         (None, 296, 246, 32)      0         
                                                                 
 flatten_1 (Flatten)         (None, 2330112)           0         
                                                                 
 dense_2 (Dense)             (None, 64)                149127232 
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 4)                 260       
                                                                 
Total params: 149,233,220
Trainable params: 149,233,22

In [33]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)   
callbacks = [earlystop]



In [34]:

history = model.fit(x = X_train, y = y_train, epochs=TrainEpochs, batch_size = 8 , shuffle=False, validation_split=0.2, callbacks=callbacks)


ValueError: Data cardinality is ambiguous:
  x sizes: 80, 80, 80, 80
  y sizes: 4
Make sure all arrays contain the same number of samples.

In [ ]:


model.save(SaveModelFolder)


In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(cf.rootfolder + 'ModelAccuracy.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(cf.rootfolder + 'ModelLoss.png')
plt.show()
